### Spark HW2 Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

%matplotlib inline is not supported in Databricks.
You can display matplotlib figures using display(). For an example, see https://docs.databricks.com/user-guide/visualizations/matplotlib-and-ggplot.html

In [3]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
ratings_df.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [9]:
links_df.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [10]:
tags_df.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [11]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [12]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

## Part 1: Spark SQL and OLAP

In [14]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Q1: The number of Users

In [16]:
%sql select count(distinct userID) from ratings

count(DISTINCT userID)
610


### Q2: The number of Movies

In [18]:
%sql select count(distinct movieID) from movies

count(DISTINCT movieID)
9742


### Q3:  How many movies are rated by users? List movies not rated before

In [20]:
%sql select title, genres from movies where movieID not in (select distinct movieID from ratings)

title,genres
"Innocents, The (1961)",Drama|Horror|Thriller
Niagara (1953),Drama|Thriller
For All Mankind (1989),Documentary
"Color of Paradise, The (Rang-e khoda) (1999)",Drama
I Know Where I'm Going! (1945),Drama|Romance|War
"Chosen, The (1981)",Drama
"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
Scrooge (1970),Drama|Fantasy|Musical
Proof (1991),Comedy|Drama|Romance
"Parallax View, The (1974)",Thriller


In [21]:
%sql select count(distinct movieID) from ratings 

count(DISTINCT movieID)
9724


### Q4: List Movie Genres

In [23]:
%sql select distinct genres from movies

genres
Comedy|Horror|Thriller
Adventure|Sci-Fi|Thriller
Action|Adventure|Drama|Fantasy
Action|Drama|Horror
Action|Animation|Comedy|Sci-Fi
Animation|Children|Drama|Musical|Romance
Action|Adventure|Drama
Adventure|Sci-Fi
Documentary|Musical|IMAX
Adventure|Children|Fantasy|Sci-Fi|Thriller


In [24]:
%sql select distinct explode(split(genres, '[|]')) as Genres from movies order by Genres 

Genres
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


### Q5: Movie for Each Category

In [26]:
%sql select title, explode(split(genres, '[|]')) as Genres from movies order by Genres 

title,Genres
Too Funny to Fail: The Life and Death of The Dana Carvey Show (2017),(no genres listed)
Superfast! (2015),(no genres listed)
Ali Wong: Baby Cobra (2016),(no genres listed)
Serving in Silence: The Margarethe Cammermeyer Story (1995),(no genres listed)
A Christmas Story Live! (2017),(no genres listed)
Green Room (2015),(no genres listed)
A Midsummer Night's Dream (2016),(no genres listed)
The Forbidden Dance (1990),(no genres listed)
Ben-hur (2016),(no genres listed)
The Brand New Testament (2015),(no genres listed)


In [27]:
%sql select a.Genres, count(*) as Count from (select explode(split(genres, '[|]')) as Genres from movies) as a group by a.Genres order by a.Genres

Genres,Count
(no genres listed),34
Action,1828
Adventure,1263
Animation,611
Children,664
Comedy,3756
Crime,1199
Documentary,440
Drama,4361
Fantasy,779


## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [29]:
ratings_df.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [30]:
movie_ratings0=ratings_df.drop('timestamp')

In [31]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings0 = movie_ratings0.withColumn("userId", movie_ratings0["userId"].cast(IntegerType()))
movie_ratings0 = movie_ratings0.withColumn("movieId", movie_ratings0["movieId"].cast(IntegerType()))
movie_ratings0 = movie_ratings0.withColumn("rating", movie_ratings0["rating"].cast(FloatType()))

In [32]:
movie_ratings0.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
 1| 70| 3.0|
 1| 101| 5.0|
 1| 110| 4.0|
 1| 151| 5.0|
 1| 157| 5.0|
 1| 163| 5.0|
 1| 216| 5.0|
 1| 223| 3.0|
 1| 231| 5.0|
 1| 235| 4.0|
 1| 260| 5.0|
 1| 296| 3.0|
 1| 316| 3.0|
 1| 333| 5.0|
 1| 349| 4.0|
+------+-------+------+
only showing top 20 rows

In [33]:
from pyspark.sql.functions import monotonically_increasing_id
movie_ratings0 = movie_ratings0.withColumn("id", monotonically_increasing_id())
movie_ratings0.show()

+------+-------+------+---+
userId|movieId|rating| id|
+------+-------+------+---+
 1| 1| 4.0| 0|
 1| 3| 4.0| 1|
 1| 6| 4.0| 2|
 1| 47| 5.0| 3|
 1| 50| 5.0| 4|
 1| 70| 3.0| 5|
 1| 101| 5.0| 6|
 1| 110| 4.0| 7|
 1| 151| 5.0| 8|
 1| 157| 5.0| 9|
 1| 163| 5.0| 10|
 1| 216| 5.0| 11|
 1| 223| 3.0| 12|
 1| 231| 5.0| 13|
 1| 235| 4.0| 14|
 1| 260| 5.0| 15|
 1| 296| 3.0| 16|
 1| 316| 3.0| 17|
 1| 333| 5.0| 18|
 1| 349| 4.0| 19|
+------+-------+------+---+
only showing top 20 rows

In [34]:
movie_ratings0.count()

Out[17]: 100836

In [35]:
from pyspark.sql.functions import col
(movie_ratings, rest) = movie_ratings0.randomSplit([0.1, 0.9])
movie_ratings.count()

Out[18]: 10057

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [37]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [38]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])
training.show()

+------+-------+------+---+
userId|movieId|rating| id|
+------+-------+------+---+
 1| 50| 5.0| 4|
 1| 673| 3.0| 39|
 1| 1298| 5.0| 90|
 1| 1408| 3.0| 94|
 1| 2596| 5.0|168|
 1| 2616| 4.0|169|
 1| 2797| 4.0|182|
 1| 3034| 5.0|199|
 2| 333| 4.0|233|
 2| 86345| 4.0|249|
 2| 91658| 2.5|252|
 3| 1587| 4.5|275|
 3| 70946| 5.0|298|
 4| 351| 3.0|325|
 4| 910| 5.0|353|
 4| 919| 5.0|356|
 4| 1060| 2.0|363|
 4| 1197| 5.0|377|
 4| 1199| 2.0|379|
 4| 1211| 1.0|381|
+------+-------+------+---+
only showing top 20 rows

In [39]:
training.count()

Out[21]: 8080

In [40]:
#Create ALS model 
als = ALS(regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

In [41]:
type(als)

Out[23]: pyspark.ml.recommendation.ALS

In [42]:
#Tune model using ParamGridBuilder
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [5, 10]) \
            .addGrid(als.maxIter, [5, 10]) \
            .addGrid(als.regParam, [0.01, 0.1]) \
            .build()
print(len(param_grid))

8

In [43]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [44]:
# Build Cross validation b
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds = 5)

In [45]:
#Fit ALS model to training data
cvModel=cv.fit(training)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [46]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = cvModel.bestModel

### Model testing
And finally, make a prediction and check the testing error.

In [48]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [49]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank: " + str(best_model.rank)), 
print (" MaxIter: " + str(best_model
    ._java_obj     # Get Java object
    .parent()      # Get parent (ALS estimator)
    .getMaxIter())), 
print (" RegParam: " + str(best_model._java_obj.parent().getRegParam())), 

RMSE = 1.3501344809650004
**Best Model**
 Rank: 10
 MaxIter: 10
 RegParam: 0.1
Out[30]: (None,)

In [50]:
predictions.show()

+------+-------+------+-----+----------+
userId|movieId|rating| id|prediction|
+------+-------+------+-----+----------+
 542| 1580| 3.5|84436| 3.9162242|
 560| 1580| 4.0|86184| 3.2760768|
 354| 1580| 3.5|53654| 3.1186962|
 115| 1645| 4.0|18200| 3.3766272|
 28| 3175| 1.5| 4394| 2.1234705|
 590| 3175| 2.5|90913| 2.2936256|
 135| 3175| 4.0|20557| 3.3230414|
 132| 8638| 4.0|20214| 1.776117|
 298| 36525| 2.5|45034| 1.6030469|
 414| 540| 2.0|62534| 1.72446|
 580| 858| 4.5|89377| 4.577468|
 222| 858| 4.5|32622| 3.9388347|
 122| 858| 5.0|18861| 4.8584895|
 279| 858| 4.0|41084| 4.3723955|
 590| 858| 5.0|90609| 3.3115263|
 135| 858| 4.0|20392| 3.9662604|
 387| 1127| 4.5|59389| 2.0397623|
 591| 1127| 3.0|91251| 1.8349891|
 222| 1721| 2.5|32656| 2.8459404|
 280| 1721| 4.0|41286| 2.1165035|
+------+-------+------+-----+----------+
only showing top 20 rows

### Model apply and see the performance

In [52]:
alldata = best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))
alldata.count()

RMSE = 0.5644455121358646
Out[32]: 9613

In [53]:
alldata.registerTempTable("alldata")

In [54]:
%sql select * from alldata order by id

userId,movieId,rating,id,prediction
1,50,5.0,4,4.8139777
1,673,3.0,39,3.0140967
1,1298,5.0,90,4.787555
1,1408,3.0,94,3.1122477
1,2596,5.0,168,4.868236
1,2616,4.0,169,3.8497894
1,2628,4.0,171,1.4667412
1,2797,4.0,182,4.071091
1,3034,5.0,199,4.866223
1,3053,5.0,201,2.9049928


In [55]:
%sql select * from movies join alldata on movies.movieId=alldata.movieId order by id

movieId,title,genres,userId,movieId,rating,id,prediction
50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,50,5.0,4,4.8139777
673,Space Jam (1996),Adventure|Animation|Children|Comedy|Fantasy|Sci-Fi,1,673,3.0,39,3.0140967
1298,Pink Floyd: The Wall (1982),Drama|Musical,1,1298,5.0,90,4.787555
1408,"Last of the Mohicans, The (1992)",Action|Romance|War|Western,1,1408,3.0,94,3.1122477
2596,SLC Punk! (1998),Comedy|Drama,1,2596,5.0,168,4.868236
2616,Dick Tracy (1990),Action|Crime,1,2616,4.0,169,3.8497894
2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Sci-Fi,1,2628,4.0,171,1.4667412
2797,Big (1988),Comedy|Drama|Fantasy|Romance,1,2797,4.0,182,4.071091
3034,Robin Hood (1973),Adventure|Animation|Children|Comedy|Musical,1,3034,5.0,199,4.866223
3053,"Messenger: The Story of Joan of Arc, The (1999)",Drama|War,1,3053,5.0,201,2.9049928


## Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [57]:
#recommend top 3 movies for each user
rec = best_model.recommendForAllUsers(3)
rec.show()

+------+--------------------+
userId| recommendations|
+------+--------------------+
 471|[[858, 3.6844938]...|
 463|[[1968, 4.37465],...|
 496|[[1214, 4.353657]...|
 148|[[40629, 4.786015...|
 540|[[1242, 5.2324915...|
 392|[[1242, 5.328126]...|
 243|[[5418, 5.380747]...|
 31|[[3153, 5.034675]...|
 516|[[1242, 4.015292]...|
 580|[[8371, 4.9335184...|
 251|[[1204, 5.335689]...|
 451|[[247, 4.3191214]...|
 85|[[66934, 4.112015...|
 137|[[1299, 4.8460994...|
 65|[[3972, 5.1979895...|
 458|[[1358, 5.599806]...|
 481|[[1729, 3.9102905...|
 53|[[4019, 4.941082]...|
 255|[[1194, 4.7605963...|
 588|[[1242, 4.350664]...|
+------+--------------------+
only showing top 20 rows

In [58]:
rec.count()

Out[35]: 595

In [59]:
rec.where(rec.userId == 575).collect()

Out[36]: [Row(userId=575, recommendations=[Row(movieId=2568, rating=3.916980743408203), Row(movieId=60756, rating=3.2804129123687744), Row(movieId=1721, rating=3.0776519775390625)])]

In [60]:
%sql select * from movies where movieId in (2568, 60756, 1721) order by movieId

movieId,title,genres
1721,Titanic (1997),Drama|Romance
2568,"Mod Squad, The (1999)",Action|Crime
60756,Step Brothers (2008),Comedy


In [61]:
rec.where(rec.userId == 232).collect()

Out[37]: [Row(userId=232, recommendations=[Row(movieId=1214, rating=4.8962788581848145), Row(movieId=2959, rating=4.895084381103516), Row(movieId=2924, rating=4.895073890686035)])]

In [62]:
%sql select * from movies where movieId in (1214, 2959, 2924) order by movieId

movieId,title,genres
1214,Alien (1979),Horror|Sci-Fi
2924,Drunken Master (Jui kuen) (1978),Action|Comedy
2959,Fight Club (1999),Action|Crime|Drama|Thriller


## Find the similar movies for movies with id: 463, 471
You can find the similar moives based on the ALS results

In [64]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors, VectorUDT

In [65]:
%sql select * from movies where movieId = 471

movieId,title,genres
471,"Hudsucker Proxy, The (1994)",Comedy


In [66]:
a = best_model.itemFactors
display(a.cache())

id,features
10,"List(-0.17399456, -0.11281259, 0.8569175, -0.54631084, -0.55400944, 1.4655101, -0.64992434, 0.11219682, -0.58871645, -0.805032)"
20,"List(0.48783374, 0.26239246, 1.025266, -0.8355324, -0.64349926, 1.1789007, -0.38052866, 0.12471497, -0.36214808, 0.46206874)"
40,"List(0.011918607, -0.6823519, 1.2163125, 0.6471562, -1.8250132, -0.11542535, -1.0401864, -0.16205989, 0.13153061, -0.6745505)"
50,"List(0.41380128, 0.52764153, 0.3219214, -0.59345657, -0.8761349, 1.5146528, -0.88337696, 0.8208594, -1.0010954, -0.6537329)"
60,"List(-0.13563386, 0.9672694, 0.74212706, -0.73763955, -0.8961581, 1.0795335, -0.14413597, 0.6976324, -0.31207517, -0.4542721)"
70,"List(0.1868956, 0.11975827, 0.24218467, 0.28644493, -1.4317477, 1.8521532, -1.1187931, 0.25387195, -0.060563233, -0.38064763)"
100,"List(0.3087319, 0.5037343, 0.49833703, -0.14110358, -0.7671431, 0.92059064, 0.051934868, 0.93045247, 0.36248496, -0.062180847)"
110,"List(0.60796386, 0.70030755, 1.4684035, 0.12191183, -0.30424312, 1.4313833, -0.60322106, 0.3105281, -0.90391064, -0.6063106)"
140,"List(-0.4878867, 0.6201137, -0.76593447, -0.5902814, 0.33427775, 0.33656263, -1.7366278, 0.20049278, 0.20113528, -1.1813973)"
150,"List(0.5027478, -0.07749295, 0.5137244, -0.25358272, -1.2210498, 1.7142764, -0.7389047, 0.6288325, 0.05641706, -1.2028278)"


In [67]:
a.where(a.id == 471).collect()

Out[40]: [Row(id=471, features=[0.41469210386276245, -0.00914978887885809, 1.2196894884109497, -1.4785195589065552, 0.542229175567627, 1.3933889865875244, 0.11748117208480835, 0.44705721735954285, 0.40512022376060486, 0.3802637755870819])]

In [68]:
a.where(a.id == 463).collect()

Out[41]: []

In [69]:
a.registerTempTable("itemMatrix")

In [70]:
%sql
select features from itemMatrix where id = 471

features
"List(0.4146921, -0.009149789, 1.2196895, -1.4785196, 0.5422292, 1.393389, 0.11748117, 0.44705722, 0.40512022, 0.38026378)"


In [71]:
%sql
select features from itemMatrix where id = 463

features


In [72]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",seed=12345, bucketLength=1.0)
a.printSchema()
#change features columns into dense vector
to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
data = a.select("id", to_vector("features").alias("features"))
data.printSchema()

root
-- id: integer (nullable = false)
-- features: array (nullable = true)
 |-- element: float (containsNull = false)

root
-- id: integer (nullable = false)
-- features: vector (nullable = true)

In [73]:
model = brp.fit(data)
model.transform(data)

Out[59]: DataFrame[id: int, features: vector, hashes: array<vector>]

In [74]:
model.approxNearestNeighbors(data, Vectors.dense([0.4146921,-0.009149789,1.2196895,-1.4785196,0.5422292,1.393389,0.11748117,0.44705722,0.40512022,0.38026378]), 6).collect()

Out[62]: [Row(id=471, features=DenseVector([0.4147, -0.0091, 1.2197, -1.4785, 0.5422, 1.3934, 0.1175, 0.4471, 0.4051, 0.3803]), hashes=[DenseVector([0.0])], distCol=5.157211161170817e-08),
 Row(id=100383, features=DenseVector([0.4584, 0.1605, 0.6619, -1.1362, 0.9767, 1.6138, -0.2073, 0.716, -0.249, 0.2862]), hashes=[DenseVector([0.0])], distCol=1.1449081505244467),
 Row(id=4226, features=DenseVector([0.0375, 0.1906, 1.0406, -1.1876, 0.1527, 1.9812, -0.3894, 0.9659, 0.082, 0.3408]), hashes=[DenseVector([0.0])], distCol=1.1950648132422377),
 Row(id=1500, features=DenseVector([1.0099, 0.0422, 0.6341, -1.4007, -0.1794, 1.5966, 0.3951, 0.4336, 0.0372, 0.4171]), hashes=[DenseVector([0.0])], distCol=1.2173377910514922),
 Row(id=5957, features=DenseVector([0.3469, 0.1886, 0.6857, -1.1686, -0.0978, 1.6653, -0.3737, 0.3267, -0.1077, 0.1119]), hashes=[DenseVector([0.0])], distCol=1.2244565522772075),
 Row(id=246, features=DenseVector([0.6531, 0.2292, 1.4375, -1.4201, -0.3641, 1.5474, 0.0866, 0.5251, -0.3328, 0.4547]), hashes=[DenseVector([0.0])], distCol=1.2517385838135278)]

In [75]:
%sql
select * from movies
where movieId in (100383,4226,1500,5957,246)

movieId,title,genres
246,Hoop Dreams (1994),Documentary
1500,Grosse Pointe Blank (1997),Comedy|Crime|Romance
4226,Memento (2000),Mystery|Thriller
5957,Two Weeks Notice (2002),Comedy|Romance
100383,Side Effects (2013),Crime|Drama|Mystery|Thriller


## Write the report 
motivation
1. step1
2. step2
3. step3
4. step4  
output and conclusion

## Movie Recommendation with Spark ALS
1. Use Alternating Least Square with Spark to predict ratings for movies in MovieLens small dataset
2. Perform Data ETL and Data Exploration to learn basic information and prepare data for analysis
3. Analyze datasets in terms of movies, users, ratings and genres via Spark SQL and OLAP
4. Train an ALS model on a dataset of 10000 data points, tune hyperparameters using cross validation to get best model, use best model to predict ratings and achieve 0.56 RMSE, recommend top 3 movies to users, and find similar movies for movies using ALS’s item matrix and k-nearest neighbors